In [1]:
import h5py
import numpy as np
import os
import imageio
import cv2
import math
import tensorflow as tf
import scipy
import scipy.io
import scipy.misc
import skimage
import skimage.io
import skimage.transform
from functools import lru_cache
from tqdm import tqdm as tqdm
from vectormath import Vector2
from model import Model

In [2]:
class H36Mparser(object):
    data = [
        'S',
        'center',
        'part',
        'scale',
        'zind',
    ]
    
    def __init__(self):
        self.annotation = h5py.File('C:/Users/I.B. Park/Desktop/h36m/annot/train.h5', 'r')
        self.image_name = np.genfromtxt('C:/Users/I.B. Park/Desktop/h36m/annot/train_images.txt', dtype = str)
        self.shuffle = np.arange(len(self))
    
    @lru_cache(maxsize=1)
    def __len__(self):
        return len(self.image_name)
    
    def __iter__(self):
        self.index = 0
        np.random.shuffle(self.shuffle)
        return self
    
    def __next__(self):
        if self.index >= len(self):
            raise StopIteration
        index = self.shuffle[self.index]
        self.index = self.index + 1
        return [self.annotation[tag][index] for tag in H36Mparser.data] + [self.image_name[index]]

In [3]:
@lru_cache(maxsize=32)
def gaussian(size, sigma=0.25, mean=0.5):
    width = size
    heigth = size
    amplitude = 1.0
    sigma_u = sigma
    sigma_v = sigma
    mean_u = mean * width + 0.5
    mean_v = mean * heigth + 0.5

    over_sigma_u = 1.0 / (sigma_u * width)
    over_sigma_v = 1.0 / (sigma_v * heigth)

    x = np.arange(0, width, 1, np.float32)
    y = x[:, np.newaxis]

    du = (x + 1 - mean_u) * over_sigma_u
    dv = (y + 1 - mean_v) * over_sigma_v

    return amplitude * np.exp(-0.5 * (du * du + dv * dv))


def generateHeatmap(size, y0, x0, sigma=1):
    pad = 3 * sigma
    y0, x0 = int(y0), int(x0)
    dst = [max(0, y0 - pad), max(0, min(size, y0 + pad + 1)), max(0, x0 - pad), max(0, min(size, x0 + pad + 1))]
    src = [-min(0, y0 - pad), pad + min(pad, size - y0 - 1) + 1, -min(0, x0 - pad), pad + min(pad, size - x0 - 1) + 1]

    heatmap = np.zeros([size, size])
    g = gaussian(3 * 2 * sigma + 1)
    heatmap[dst[0]:dst[1], dst[2]:dst[3]] = g[src[0]:src[1], src[2]:src[3]]

    return heatmap


def cropImage(image, center, scale, rotate, resolution):
    center = Vector2(center)  # assign new array
    height, width, _ = image.shape
    crop_ratio = 200 * scale / resolution
    if crop_ratio >= 2:  # if box size is greater than two time of resolution px
        # scale down image
        height = math.floor(height / crop_ratio)
        width = math.floor(width / crop_ratio)

        if max([height, width]) < 2:
            # Zoomed out so much that the image is now a single pixel or less
            raise ValueError("Width or height is invalid!")

        image = skimage.transform.resize(image, (height, width), mode='constant')
        center /= crop_ratio
        scale /= crop_ratio

    ul = (center - 200 * scale / 2).astype(int)
    br = (center + 200 * scale / 2).astype(int)  # Vector2

    if crop_ratio >= 2:  # force image size 256 x 256
        br -= (br - ul - resolution)

    pad_length = math.ceil((ul - br).length - (br.x - ul.x) / 2)

    if rotate != 0:
        ul -= pad_length
        br += pad_length

    src = [max(0, ul.y), min(height, br.y), max(0, ul.x), min(width, br.x)]
    dst = [max(0, -ul.y), min(height, br.y) - ul.y, max(0, -ul.x), min(width, br.x) - ul.x]

    new_image = np.zeros([br.y - ul.y, br.x - ul.x, 3], dtype=np.float64)
    new_image[dst[0]:dst[1], dst[2]:dst[3], :] = image[src[0]:src[1], src[2]:src[3], :]

    if rotate != 0:
        new_image = skimage.transform.rotate(new_image, rotate)
        new_height, new_width, _ = new_image.shape
        new_image = new_image[pad_length:new_height - pad_length, pad_length:new_width - pad_length, :]

    if crop_ratio < 2:
        new_image = skimage.transform.resize(new_image, (resolution, resolution), mode='constant')

    return new_image

In [4]:
def generateVoxel(voxel_xy_resolution, voxel_z_resolution, xy, z, heatmap_xy_coefficient, heatmap_z_coefficient):
    volume = np.ndarray(shape = (voxel_xy_resolution, voxel_xy_resolution, voxel_z_resolution), dtype = np.float64)
    xy_view = generateHeatmap(size = voxel_xy_resolution, y0 = xy[1], x0 = xy[0], sigma = heatmap_xy_coefficient)
    z_view = gaussian(heatmap_z_coefficient)[math.ceil(heatmap_z_coefficient/2) - 1]
    cnt = 0
    for i in range(z - math.floor(heatmap_z_coefficient/2), z + math.floor(heatmap_z_coefficient/2) + 1):
        if 0 <= i < voxel_z_resolution:
            volume[:, :, i] = z_view[cnt] * xy_view
        cnt = cnt + 1
    return volume

In [5]:
def decode_image_name(image_name):
    subject_action, camera_frame, _ = image_name.split('.')
    split = subject_action.split('_')
    subject = split[0]
    action = split[1]
    if len(split) >= 3:
        action = action + '_' + split[2]
    camera, frame = camera_frame.split('_')
    
    return subject, action, camera, frame

In [6]:
parser = H36Mparser()

z_limits = np.squeeze(scipy.io.loadmat('C:/Users/I.B. Park/Desktop/c2f-vol-demo-master/matlab/utils/data/voxel_limits.mat')['limits'])
z_centers = (z_limits[1:65] + z_limits[0:64]) / 2
z_delta = z_limits[32]

In [7]:
data_dir = 'D:/data/Human3.6M/converted'

# Voxel heatmap sigma.
heatmap_xy_coefficient = 2
heatmap_z_coefficient = -1

# Voxel resolution.
voxel_xy_resolution = 64
voxel_z_fine_resolution = 64
voxel_z_coarse_resolution = -1

voxel_z_resolutions = [1, 2, 4, voxel_z_fine_resolution]

In [8]:
model = Model(features = 256, joints = len(parser.annotation['zind'][0]), z_resolutions = voxel_z_resolutions)

In [9]:
'''
print(model.z_resolutions)
for stage, _ in enumerate(model.z_resolutions):
    print(model.voxel_groundtruth[stage], model.voxels[stage])
'''

'\nprint(model.z_resolutions)\nfor stage, _ in enumerate(model.z_resolutions):\n    print(model.voxel_groundtruth[stage], model.voxels[stage])\n'

In [10]:
# with tf.Session() as sess:
global_step = tf.Variable(0, trainable=False, name='global_step')

with tf.variable_scope('loss'):
    loss = sum([
        tf.losses.mean_squared_error(
            tf.reshape(model.voxel_groundtruth[stage], shape=[-1, 64 * 64, model.joints * z_resolution]),
            tf.reshape(model.voxels[stage], shape=[-1, 64 * 64, model.joints * z_resolution]))
        for stage, z_resolution in enumerate(model.z_resolutions)
    ])

    tf.summary.scalar('loss', loss)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.RMSPropOptimizer(name='optimizer', learning_rate=2.5e-4).minimize(loss, global_step=global_step)

with tf.Session() as sess:
    summary_merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('log', sess.graph)
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver(max_to_keep=10)

    # if MODEL.pretrained.is_using:
    #     saver.restore(sess, os.path.join(MODEL.pretrained.path, '%s.ckpt' % file_name))


    tf.train.global_step(sess, global_step)
    for epoch in range(3):
        with tqdm(range(len(parser))) as progress:
            progress.set_description('Epoch(%d)' % epoch)

            for S, center, part, scale, zind, image_name in parser:
                voxels = list()

                # Extract subject and camera name from an image name.
                subject, _, camera, _ = decode_image_name(image_name)

                # Pre-calculate constants.
                image_xy_resolution = 200 * scale

                # Crop RGB image.
                image = skimage.io.imread('%s/%s-2/%s' % (data_dir, subject, image_name))
                image = cropImage(image, center, scale, 0, 256)
                # imageio.imwrite('rgb.png', image)

                # Build voxel.
                for voxel_z_coarse_resolution in voxel_z_resolutions:
                    # heatmap_z_coefficient is 1, 1, 1, 3, 5, 7, 13 for 1, 2, 4, 8, 16, 32, 64.
                    heatmap_z_coefficient = 2 * math.floor((6 * heatmap_xy_coefficient * voxel_z_coarse_resolution / voxel_z_fine_resolution + 1)/2) + 1

                    # Convert the coordinate from a RGB image to a cropped RGB image.
                    xy = voxel_xy_resolution * (part - center) / image_xy_resolution + voxel_xy_resolution * 0.5

                    voxel = np.ndarray(shape = (voxel_xy_resolution, voxel_xy_resolution, len(part) * voxel_z_coarse_resolution))
                    for part_idx in range(len(part)):
                        # zind range (1, 64)
                        # z range (0, 63)
                        z = math.ceil(zind[part_idx] * voxel_z_coarse_resolution / voxel_z_fine_resolution) - 1
                        voxel[:, :, part_idx * voxel_z_coarse_resolution : (part_idx + 1) * voxel_z_coarse_resolution] = generateVoxel(
                            voxel_xy_resolution, voxel_z_coarse_resolution,
                            xy[part_idx], z,
                            heatmap_xy_coefficient, heatmap_z_coefficient)
                    voxels.append(voxel)

                    '''
                    for z in range(voxel_z_coarse_resolution):
                        for y in range(voxel_xy_resolution):
                            for x in range(voxel_xy_resolution):
                                voxel[y, x, z] = np.max(voxel[y, x, [part * voxel_z_coarse_resolution + z for part in range(len(part))]])
                    for z in range(voxel_z_coarse_resolution):
                        imageio.imwrite('%02d_%02d.png' % (voxel_z_coarse_resolution, z), voxel[:, :, z])
                    '''

                train_images = np.ndarray(shape=(1, 256, 256, 3), dtype = np.float64)
                train_voxels = [np.ndarray(shape=(1, 64, 64, len(parser.annotation['zind'][0]) * z_resolution), dtype = np.float64)
                                for z_resolution in voxel_z_resolutions]
                train_images[0, :, :, :] = image
                for stage in range(len(voxel_z_resolutions)):
                    train_voxels[stage][0, :, :, :] = voxels[stage]

                feed = dict()
                feed[model.images] = train_images
                for stage in range(len(voxel_z_resolutions)):
                    feed[model.voxel_groundtruth[stage]] = train_voxels[stage]
                feed[model.is_training] = True



                _, result, summary, step = sess.run([optimizer, loss, summary_merged, tf.train.get_global_step()],
                                                    feed_dict=feed)
                progress.set_postfix(loss=result)
                progress.set_description("epoch: %d" % epoch)
                progress.update(1)

                writer.add_summary(summary, step)

        save_path = saver.save('save_%03d.ckpt' % (math.round(step / len(parser))))

  0%|                                                                                                            | 0/312188 [00:00<?, ?it/s]


InvalidArgumentError: Tensor must be 4-D with last dim 1, 3, or 4, not [1,64,17]
	 [[Node: voxel_1D_01 = ImageSummary[T=DT_FLOAT, bad_color=Tensor<type: uint8 shape: [4] values: 255 0 0...>, max_images=3, _device="/job:localhost/replica:0/task:0/cpu:0"](voxel_1D_01/tag, strided_slice/_423)]]

Caused by op 'voxel_1D_01', defined at:
  File "c:\users\i.b. park\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\i.b. park\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\i.b. park\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\i.b. park\appdata\local\programs\python\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\i.b. park\appdata\local\programs\python\python35\lib\sit